In [3]:
from pyspark.sql import SparkSession
from textblob import TextBlob
from pyspark.sql.functions import udf,col
from pyspark.sql.types import StringType, FloatType
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


In [4]:
spark = SparkSession \
        .builder \
        .master('spark://10.10.28.172:7077') \
        .appName('pre_processing') \
        .enableHiveSupport() \
        .config("spark.pyfiles", "preproc_functions.py") \
        .getOrCreate()

# Preventing errors of having too many settings in hive-site.xml
spark.sparkContext.setLogLevel('OFF')
#spark.sparkContext.addPyFile("preproc_functions.py")
# name of database
spark.sql('use twitter_data')

22/04/09 20:15:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


+------------+
|   namespace|
+------------+
|     default|
|twitter_data|
+------------+



In [11]:
query = 'SELECT * FROM processed_data'

df = spark.sql(query)

In [12]:
#text = "Hello I lve the news!"

def sentiment_tblob(text):
    return TextBlob(text).sentiment.polarity

# Vader is twice as fast as textblob...
def sentiment_vader(text):
    analyzer = SentimentIntensityAnalyzer()
    return analyzer.polarity_scores(text)
    
udf_sentiment = udf(lambda text: sentiment_tblob(text), FloatType())
#sentiment_scores = df.select('id','text','created_at').withColumn('sentiment_score', udf_sentiment(col('text')))
#sentiment_scores.show()
sentiment_score = df.withColumn('sentiment',udf_sentiment('text')).select('id','created_at','sentiment','text').orderBy('created_at',ascending=False)

In [13]:
sentiment_score.write.mode("overwrite").saveAsTable("twitter_data.sentiment_results")